In [ ]:
import powerplantmatching as pm
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

import matplotlib
matplotlib.use('Agg')

In [ ]:
df_init = pm.powerplants()
nuclear = df_init.query('Fueltype=="Nuclear"')
df_init = df_init.query('Fueltype!="Nuclear"')
df_init = df_init.append(nuclear)

In [ ]:
#%% powerplantmatching plot
stepsize=4;

count = 0;
for k in range(len(df_init)-len(nuclear), len(df_init), stepsize):
        df = df_init[0:k]

        df = df.powerplant.set_uncommon_fueltypes_to_other()
        shown_fueltypes = df.Fueltype.unique()
        df = df[df.lat.notnull()]
        df = df.query('-10 <= lon <= 30').query('36 <= lat <= 70')

        fig, ax = plt.subplots(figsize=(5,5), subplot_kw={"projection": ccrs.EqualEarth()})

        scale=1e-2
        ax.scatter(
            df.lon,
            df.lat,
            s=df.Capacity * scale,
            c=df.Fueltype.map(pm.get_config()["fuel_to_color"]),
            edgecolor="face",
            facecolor="face",
            alpha=1,
            zorder=3,
            transform=ccrs.PlateCarree(),
        )
        ax.scatter([-10, 40, -10, 40, -10], [36, 36, 70, 70, 36],
                   transform=ccrs.PlateCarree(), color='white')
        ax.coastlines(linewidth=0.3, zorder=1)
        ax.spines['geo'].set_visible(False)

        fig.savefig(f'../powerplantmatching-{count}.png', bbox_inches='tight', dpi=80)
        print(f'powerplantmatching-{count}.png')
        count += 1

In [ ]:
from PIL import Image
import numpy as np

def gen_frame(path):
    im = Image.open(path)
    alpha = im.getchannel('A')

    # Convert the image into P mode but only use 255 colors in the palette out of 256
    im = im.convert('RGB').convert('P', palette=Image.ADAPTIVE, colors=255)

    # Set all pixel values below 128 to 255 , and the rest to 0
    mask = Image.eval(alpha, lambda a: 0 if a <=128 else 0)

    # Paste the color of index 255 and use alpha as a mask
    im.paste(0, mask)

    # The transparency index is 255
    im.info['transparency'] = 0
    return im


im1 = gen_frame(f'../powerplantmatching-{count-1}.png')
append_images = np.append([f"../powerplantmatching-{i}.png" for i in range(count-1,-1,-1)],
                          [f"../powerplantmatching-{i}.png" for i in range(0,count)])

im1.save('../gifs/powerplantmatching.gif', save_all=True, append_images=[gen_frame(append_images[i]) for i in range(len(append_images))],
         disposal=2, loop=0, duration=50)
im1.save('../powerplantmatching.png', save_all=False, append_images=[],
         disposal=2, duration=50)

In [ ]:
# delete images...
import os
for file in [f"../powerplantmatching-{i}.png" for i in range(count)]:
    os.remove(file) 